In [21]:
import argparse
import logging
import os
import torch
from torch.utils.data import DataLoader, random_split
import json
import rdkit.Chem as Chem
#from rxntorch.models.reactivity_network import ReactivityNet as RxnNet, ReactivityTrainer as RxnTrainer
#from rxntorch.utils import collate_fn
import rdkit
from rxntorch.containers.reaction import Rxn
from rxntorch.containers.molecule import Mol
from rxntorch.containers.dataset import RxnGraphDataset as RxnGD
from rxntorch.utils import collate_fn
import warnings
warnings.filterwarnings("ignore")
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from collections import defaultdict

import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder


In [22]:
import re
import glob
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False



def get_atom_mapping_doyle(mol_dir):
    
    smiles_mapping=defaultdict(dict) 
    
    for mol_fn in glob.glob(mol_dir):
        atoms , labels, atom_mapping =[], [], []
        if True:
            
            m = Chem.MolFromMolFile(mol_fn)
            smiles=Chem.MolToSmiles(m)
            mol_lines=open(mol_fn,'r').readlines()

            for i,line in enumerate(mol_lines[4:]):
                l=re.sub(' +', ' ', line.strip('\n'))
                l2=l.split(' ')
                if len(l2)==17:
                    atom=l2[4]
                    if atom !='' and '*' not in atom and 'H' not in atom:
                        atoms.append(atom)
                if "atom_labels" in line:
                    labels=[i.strip('*') for i in mol_lines[i+1+4].strip('\n').split(' ') if ((i!='') and ('H' not in i))]
            if len(atoms)==len(labels):
                for i in range(len(atoms)):
                    atom_mapping.append((atoms[i],labels[i]))            
            else:
                print('Atoms and lables don\'t match')
                atom_mapping=[]

            if smiles not in smiles_mapping:
                smiles_mapping[mol_fn.split('/')[-1].split('.')[0]]=atom_mapping
            else:
                print("key exsists")
    return smiles_mapping



In [9]:
smiles_mapping=get_atom_mapping_doyle("data/doyle_reaction_mols/*.mol")

In [10]:
smiles_mapping

defaultdict(dict,
            {'1-bromo-4-ethylbenzene': [('C', 'C1'),
              ('C', 'C4'),
              ('C', 'C2'),
              ('C', 'C2'),
              ('C', 'C3'),
              ('C', 'C3'),
              ('C', 'C4'),
              ('C', 'C1'),
              ('Br', 'Br1')],
             '1-bromo-4-(trifluoromethyl)benzene': [('C', 'C1'),
              ('C', 'C4'),
              ('C', 'C2'),
              ('C', 'C2'),
              ('C', 'C3'),
              ('C', 'C3'),
              ('C', 'C7'),
              ('F', 'F1'),
              ('F', 'F2'),
              ('F', 'F3'),
              ('Br', 'Br1')],
             '1-bromo-4-methoxybenzene': [('C', 'C1'),
              ('C', 'C4'),
              ('C', 'C2'),
              ('C', 'C2'),
              ('C', 'C3'),
              ('C', 'C3'),
              ('O', 'O1'),
              ('C', 'C1'),
              ('Br', 'Br1')],
             '1-chloro-4-(trifluoromethyl)benzene': [('C', 'C1'),
              ('C', 'C4'),
     

In [30]:
rows=set();y_factors=set()
#data_type='az'
data_type='suzuki'
data_type='doyle'
row=set()
path = 'data/'
file_name = data_type+'_reactions_data.json'
mol_path=data_type+'_reaction_mols'

data_dict=defaultdict(lambda: defaultdict(float))
rxns = [];max_nbonds = 10   
#smiles_mapping = get_atom_mapping_doyle(mol_path+'/*.mol') 

with open(os.path.join(path, file_name)) as datafile:
    data = json.load(datafile)
    lines= data
    for line in lines:
        
        if data_type=='az':
            name= reaction_num = line["Reaction_Num"]
            r_yield=line['yield']['Yield']
        if data_type=='doyle':
            
            #plate, row, col = line["plate"], line["row"], line["col"]
            #name="-".join(map(str,[plate, row, col]))
            name= reaction_num = line["Id"]
            r_yield=line['yield']
        elif data_type=='suzuki':
            name= reaction_num = line["Id"]
            r_yield=line['yield']

        reactants=line['reactants']
        rxn = Rxn(name,reactants,r_yield)
        mol_reactants=rxn.reactants
        for mol_idx in range(len(mol_reactants)):
            current_molecule = mol_reactants[mol_idx]
            category = current_molecule.category
            vib_modes = current_molecule.vib_modes

            atoms =current_molecule.atoms
            all_attributes=current_molecule.get_attributes().squeeze().tolist()
            weight,volume,surface_area,ovality,hardness,dipole_moment,electronegativity,HOMO,LUMO=all_attributes
            data_dict[name][category] = current_molecule.name
            data_dict[name][category +'_molecular_weight'] = round(float(weight),5)
            data_dict[name][category +'_molecular_volume'] = round(float(volume),5)
            data_dict[name][category +'_surface_area'] = round(float(surface_area),5)
            data_dict[name][category +'_ovality'] = round(float(ovality),5)
            data_dict[name][category +'_hardness'] = round(float(hardness),5)
            data_dict[name][category +'_dipole_moment'] = round(float(dipole_moment),5)
            data_dict[name][category +'_electronegativity'] = round(float(electronegativity),5)
            data_dict[name][category +'_E_HOMO'] = round(float(HOMO),5)
            data_dict[name][category +'_E_LOMO'] = round(float(LUMO),5)
            for n in range(len(vib_modes)):
                data_dict[name][category + '_V'+str(n)+'_frequency'] = round(float(vib_modes[n][0]),5)
                data_dict[name][category + '_V'+str(n)+'_intensity'] = round(float(vib_modes[n][1]),5)
            
            for atom in atoms:
                if 'H' not in atom['name']: #exculding hydrogen for now
                    if 'partial_charge' in atom:
                        data_dict[name][category+'_.'+atom['name']+'_electrostatic_charge']=round(float(atom['partial_charge']),5)

                    if 'nmr_shift' in atom:
                        if data_type=='az':
                            data_dict[name][category+'_.'+atom['name']+'_NMR_shift']= round(float(atom['nmr_shift'].strip('=')),5)
                        else:
                            data_dict[name][category+'_.'+atom['name']+'_NMR_shift']= round(float(atom['nmr_shift']),5)
        if isfloat(r_yield):                
            data_dict[name]['yield'] = round(float(r_yield),5)      
        else:
            del data_dict[name]
            rows.add(name)
print("reactions with problematic yield: ",len(rows))

reactions with problematic yield:  8


In [44]:
isfloat(lines[741]['yield'])

False

In [45]:
lines[741]['yield']

'#VALUE!'

In [41]:
rows

{742, 787, 842, 843, 880, 975, 1059, 1240}

In [32]:

df_o=pd.DataFrame.from_dict(data_dict, orient='index')
df_o=df_o.reset_index()
print(df_o.shape)
#df.to_csv('data/doyle_data.csv')
#df_o=pd.read_csv('data/doyle_data.csv')


categorical =list( df_o.columns[(df_o.dtypes.values != np.dtype('float64'))])
to_drop=list(df_o.columns[(df_o == 0).all()])+categorical#+['additive','base','aryl_halide','ligand']
scaler = StandardScaler()


df=df_o
df.drop(to_drop, axis=1, inplace=True)
curr_cols=list(df.columns)
curr_cols.remove('yield')
new_cols=['yield']+curr_cols
len(new_cols)==len(curr_cols)
print("Number of features: ",len(list(new_cols)))
df.fillna(0, inplace=True)
df=df[new_cols]
#if data_type=='az' or data_type=='doyle':
df.iloc[:,1:] = scaler.fit_transform(df.iloc[:,1:].to_numpy())

(4132, 190)
Number of features:  165


In [33]:
set(df['Halide_molecular_weight'])

{-1.574076143491168,
 -1.2641916582161308,
 -0.7781195214673908,
 -0.6014653509735571,
 -0.2915806445994586,
 -0.20291616218442793,
 -0.18091105697233387,
 -0.1363454457052228,
 0.018304061774640595,
 0.10696876528873236,
 0.37195024212854444,
 0.5926239092071566,
 0.8580221577782231,
 1.2112754450995338,
 2.4297875287317394}

In [34]:
data_dict=df.transpose().to_dict(orient='list')

In [19]:
df.to_csv('data/'+data_type+'_data.csv')
data_dict=df.transpose().to_dict(orient='list')
with open('data/'+data_type+'_reactions_data.txt','w') as f:
    for key,val in data_dict.items():
        #if data_type=='az' or data_type=='suzuki':
        f.write(str(key)+'\t'+','.join(map(str,val[1:]))+'\n')
        #else:
            #f.write(val[1]+'\t'+','.join(map(str,val[2:]))+'\n')
    

In [20]:
len(data_dict)

180